## This Featurewiz+SimpleXGBModel Template has Received Top Scores in Multiple Hackathons - Current Ranks in the following Hackathons are:
AnalyticsVidhya Hackathons: https://datahack.analyticsvidhya.com/contest/all/
1.  Big_Mart Sales Prediction Score: 1147  -- Rank 250 out of 41,361 = That's a Top <1% Rank!!
1.  Loan Status Predictions Score 0.791  -- Rank 850 out of 67,424 - Top 1.25% Rank

Machine Hack Hackathons: https://www.machinehack.com/hackathon
1.  Machine Hack Flight Ticket Score 0.9389 -- Rank 165 out of 2723 - Top 6% Rank!
1.  Machine Hack Data Scientist Salary class Score 0.417 -- Rank 58 out of 1547 - Top 3.7% Rank! (Autoviml Score was 0.329 -- less than 0.417 of Featurewiz+Simple even though an NLP problem!)
1.  MCHACK Book Price NLP Score 0.7336 -- Rank 104 Autoviml NLP problem and should have done better


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.dates as md
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

In [ ]:
from load_kaggle import load_kaggle

In [ ]:
subm, train, test = load_kaggle()

In [ ]:
test.columns

# Remember to install featurewiz first

In [ ]:
!pip install featurewiz --upgrade 
#!python3 -m pip install git+https://github.com/AutoViML/featurewiz.git

In [ ]:
from featurewiz import FE_kmeans_resampler, FE_find_and_cap_outliers, EDA_find_outliers
from featurewiz import FE_convert_all_object_columns_to_numeric, split_data_n_ways, FE_create_categorical_feature_crosses
from featurewiz import FE_create_time_series_features, FE_concatenate_multiple_columns
from featurewiz import simple_XGBoost_model
import featurewiz as FW

In [ ]:
target = 'target'
idcols = ['id']

In [ ]:
### In regression problems, you might want to convert target into log(target)
#df[target] = (df[target] - np.mean(df[target]))/np.std(df[target])
#train[target] = np.log(train[target].values)

In [ ]:
### you can manually set features you want included in dataset here or automatically
features = test.columns.tolist()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

In [ ]:
df = train[features+[target]]
train = train[features+[target]]
print(train.shape)
train.head(1)

In [ ]:
test = test[features]
print(test.shape)
test.head(1)

In [ ]:
nums = [x for x in df.select_dtypes(include='float').columns.tolist() if x not in [target]]
nums

In [ ]:
cats = [x for x in df.select_dtypes(include='object').columns.tolist() if x not in [target]]
cats

# You can use AutoViz to first get some insights and do additional feature selection or engineering

In [ ]:
#!pip install autoviz

In [ ]:
#from autoviz.AutoViz_Class import AutoViz_Class
#AV = AutoViz_Class()
#filename = ""
#sep = ","
#dft = AV.AutoViz(
#    filename,
#    sep=",",
#    depVar=target,
#    dfte=train,
#    header=0,
#    verbose=0,
#    lowess=False,
#    chart_format="svg",
#    max_rows_analyzed=50000,
#    max_cols_analyzed=30,
#)

## These are what I learned from AutoViz Plots on the Tabular Playground Jan 2021 dataset:
Create new categorical variables based on:
cont2 into 9 clusters
cont3 into 2 clusters
cont14 into 2 clusters
cont10 into 2 clusters
cont4 into 3 clusters

Create new interaction variables based on:
cont3, cont14
cont6, cont7
cont6, cont9
cont6, cont12
cont11, cont12


Create new binning variables based on distributions:
cont1 4 modes 
cont3 3 modes
cont11 3 modes
cont13 might need 4 binning
cont9 might need 4 binning
cont 12 might need 9 bins
cont4 might need 3 bins

In-situ transform these variables:
cont12 needs log transformation
cont8 needs log transformation

## You can do some binning of numeric columns here using Featurewiz

In [ ]:
### You can plot to see how this numeric column would be good for binning
#df['cont2'].hist(bins=30);

In [ ]:
## This dictionary tells Featurewiz how many bins for each feature
bin_dict = {'cont1':4, 'cont2':9, 'cont3':3, 'cont4':3, 'cont6':3,
            'cont9':4, 'cont10':2, 'cont11':3, 
            'cont12':9, 'cont13':4, 'cont14':2}

In [ ]:
train, test = FW.FE_discretize_numeric_variables(train, bin_dict, test)

In [ ]:
binned_cols = [x for x in train.columns if x.endswith('_discrete')]
len(binned_cols)

In [ ]:
train_copy = train.copy(deep=True)
test_copy = test.copy(deep=True)

## Now create some groupby aggregates based on the binned variables

In [ ]:
### This handy function subtracts one list from another
def left_subtract(l1,l2):
    lst = []
    for i in l1:
        if i not in l2:
            lst.append(i)
    return lst

## Remember you can transform highly skewed variables using log_transform

In [ ]:
col_dict = {'cont12':'log', 'cont8':'log'}
train_copy = FW.FE_transform_numeric_columns(train_copy, col_dict)
test_copy = FW.FE_transform_numeric_columns(test_copy, col_dict)
train_copy.head(2)

## Add groupby features based on binned columns above

In [ ]:
train_copy, test_copy = FW.FE_add_groupby_features_aggregated_to_dataframe(train, agg_types=['mean', 'std'],
                                    groupby_columns=binned_cols,
                                    ignore_variables=idcols+[target], test=test)

In [ ]:
print(train_copy.shape)
train_copy.head(1)

In [ ]:
print(test_copy.shape)
test_copy.head(1)

# Now find outliers and cap them to their second highest values

In [ ]:
### Take a look at whether it makes sense to cap outliers ###
_ = FE_find_and_cap_outliers(train,[target],verbose=1)
#test = FE_find_and_cap_outliers(test,nums,verbose=0)

## In some cases it makes sense to drop training rows where target is less than a certain number - so it removes outliers

In [ ]:
train_copy = train_copy[~(train[target]<5)]
#train_copy[target].hist(bins=30);

# Now let us select the best variables using featurewiz

In [ ]:
train_best, test_best = FW.featurewiz(train_copy, 
                        target, test_data=test_copy)

In [ ]:
preds = test_best.columns.tolist()
preds

In [ ]:
preds = preds[1:]

In [ ]:
train_best = train_copy[preds+[target]]
test_best = test_copy[preds]
print(train_best.shape, test_best.shape)

## This simple XGBoost model works wonders since it is highly effective in many competitions

In [ ]:
y_preds = simple_XGBoost_model(X_XGB=train_best[preds], Y_XGB=train_best[target],
                               X_XGB_test=test_best[preds], modeltype='Regression')

In [ ]:
#subm = test[idcols]
#subm = pd.DataFrame()
subm[target] = y_preds
subm.head()

In [ ]:
 #!pip3 install --upgrade Pillow

In [ ]:
subm.to_csv(target+'_Regression_submission.csv',index=False)